In [1]:
#dependencies
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm,tqdm_notebook
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
import torch.optim as optim

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
path_of_data='D:/pytorch-nlp-notebooks/IMDB Dataset.csv/IMDB Dataset.csv'
pd.read_csv(path_of_data).sample(3) #here we print 3 rows randomly and see that the data is not encoded

,review,sentiment
34173,This movie is brilliant. The comments made bef...,positive
37937,I know not why people considered it trashy or ...,positive
2673,Growing up in the late 60s and 70s I could not...,positive


In [4]:
from sklearn.preprocessing import LabelEncoder
import re

#dataset class will deal with loading and preprocessing of the dataset
class dataset(Dataset):
    def __init__(self,path):
        corpus=[]
        le=LabelEncoder()
        df=pd.read_csv(path)
        if type(df.sentiment[0])==str:
            df.sentiment=le.fit_transform(df.sentiment)
        for i in tqdm(range(len(df.review))):
            text=df.review[i]
            text=text.lower()
            text=re.sub(r'[^a-zA-Z]{1,10}',' ',text)
            corpus.append(text)
        self.cv=CountVectorizer(stop_words='english', max_df=0.99, min_df=0.005)
        self.x=self.cv.fit_transform(corpus).toarray()
        self.labels=list(df.sentiment)
        self.token2idx = self.cv.vocabulary_
        #print('Token to index:',self.token2idx)
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
        print(self.idx2token)
        
    def __getitem__(self, i):
        return self.x[i, :], self.labels[i]
    
    def __len__(self):
        return self.x.shape[0]

In [5]:
data = dataset(path_of_data)
train_loader = DataLoader(data, batch_size=4096)
print(data[5][0].shape)

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:07<00:00, 6320.43it/s]


{2083: 'reviewers', 1596: 'mentioned', 2771: 'watching', 1366: 'just', 798: 'episode', 1485: 'll', 1208: 'hooked', 2091: 'right', 824: 'exactly', 1134: 'happened', 273: 'br', 2554: 'thing', 2417: 'struck', 2161: 'scenes', 2730: 'violence', 2216: 'set', 2828: 'word', 2634: 'trust', 1161: 'hearted', 1959: 'pulls', 730: 'drugs', 2221: 'sex', 416: 'classic', 2691: 'use', 316: 'called', 1059: 'given', 2187: 'security', 2371: 'state', 975: 'focuses', 1534: 'mainly', 411: 'city', 2186: 'section', 1919: 'prison', 1063: 'glass', 869: 'face', 1181: 'high', 762: 'em', 1204: 'home', 1317: 'irish', 599: 'death', 899: 'far', 179: 'away', 2150: 'say', 1533: 'main', 117: 'appeal', 873: 'fact', 1067: 'goes', 2252: 'shows', 2841: 'wouldn', 581: 'dare', 993: 'forget', 1909: 'pretty', 1840: 'pictures', 1535: 'mainstream', 169: 'audiences', 378: 'charm', 2113: 'romance', 692: 'doesn', 1599: 'mess', 2149: 'saw', 1681: 'nasty', 2480: 'surreal', 517: 'couldn', 2001: 'ready', 2770: 'watched', 647: 'developed',

In [6]:
#model definition
class classifier(nn.Module):
    def __init__(self,vocab_size,hidden1,hidden2):
        super(classifier,self).__init__()
        self.fc1=nn.Linear(vocab_size,hidden1)
        self.fc2=nn.Linear(hidden1,hidden2)
        self.fc3=nn.Linear(hidden2,1)
    def forward(self,inputs):
        x=F.relu(self.fc1(inputs.squeeze(1).float()))
        print(x)
        x=F.relu(self.fc2(x))
        return self.fc3(x)

In [7]:
model=classifier(len(data.token2idx), 128, 64)
model

classifier(
  (fc1): Linear(in_features=2862, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [8]:
criterion=nn.BCEWithLogitsLoss()
#print([p for p in model.parameters()])
optim=optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.001)

In [10]:
#training the model
model.train()
train_losses=[]
epochs=10
for epoch in range(epochs):
    progress_bar = tqdm_notebook(train_loader, leave=False)
    losses = []
    total = 0
    for inputs, target in progress_bar:
        model.zero_grad()

        output = model(inputs)
        loss = criterion(output.squeeze(), target.float())
        
        loss.backward()
              
        nn.utils.clip_grad_norm_(model.parameters(), 3)

        optim.step()
        
        progress_bar.set_description(f'Loss: {loss.item():.3f}')
        
        losses.append(loss.item())
        total += 1
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)
        
    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')

C:\Users\91884\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


tensor([[0.0732, 0.0000, 0.0000,  ..., 0.1030, 0.0051, 0.0464],
        [0.0535, 0.0000, 0.0000,  ..., 0.1515, 0.0868, 0.1060],
        [0.1189, 0.0000, 0.0000,  ..., 0.0718, 0.1730, 0.0833],
        ...,
        [0.0000, 0.0000, 0.1704,  ..., 0.0879, 0.0000, 0.0000],
        [0.0936, 0.1630, 0.0000,  ..., 0.0000, 0.0552, 0.0000],
        [0.2023, 0.1576, 0.0000,  ..., 0.1520, 0.0863, 0.0895]],
       grad_fn=<ReluBackward0>)
tensor([[0.0000, 0.0866, 0.0000,  ..., 0.0607, 0.0360, 0.0295],
        [0.1333, 0.0000, 0.2188,  ..., 0.0232, 0.0572, 0.0754],
        [0.0000, 0.0068, 0.0000,  ..., 0.0636, 0.0000, 0.0038],
        ...,
        [0.2235, 0.1883, 0.0000,  ..., 0.0000, 0.0391, 0.1279],
        [0.0018, 0.0000, 0.0000,  ..., 0.0000, 0.0108, 0.0000],
        [0.0070, 0.0842, 0.0677,  ..., 0.0000, 0.0581, 0.0000]],
       grad_fn=<ReluBackward0>)
tensor([[0.1142, 0.0346, 0.0000,  ..., 0.2654, 0.0651, 0.1424],
        [0.1857, 0.4368, 0.0000,  ..., 0.0000, 0.2129, 0.0836],
        [0.1

tensor([[0.6420, 0.3671, 0.1619,  ..., 0.1540, 0.0000, 0.6141],
        [0.4865, 0.1879, 0.0519,  ..., 0.0721, 0.0000, 0.5880],
        [0.4854, 0.2199, 0.0779,  ..., 0.0465, 0.0000, 0.4844],
        ...,
        [0.4354, 0.0000, 0.1207,  ..., 0.0000, 0.0000, 0.2779],
        [0.4437, 0.3800, 0.0000,  ..., 0.0000, 0.0000, 0.2339],
        [0.7833, 0.7029, 0.3112,  ..., 0.2407, 0.0000, 0.6725]],
       grad_fn=<ReluBackward0>)
tensor([[0.0761, 0.5787, 0.3284,  ..., 0.3847, 0.0000, 0.1265],
        [0.6268, 0.5010, 0.6058,  ..., 0.2707, 0.0000, 0.6179],
        [0.2772, 0.5481, 0.3047,  ..., 0.3081, 0.0000, 0.3027],
        ...,
        [1.7505, 0.7183, 0.1677,  ..., 0.0000, 0.0000, 1.7284],
        [0.9059, 0.1228, 0.0000,  ..., 0.0000, 0.0000, 0.7720],
        [0.2332, 0.1473, 0.0870,  ..., 0.0000, 0.0000, 0.1606]],
       grad_fn=<ReluBackward0>)
tensor([[0.6273, 0.3381, 0.0814,  ..., 0.3432, 0.0000, 0.6684],
        [1.1635, 1.5302, 0.7455,  ..., 0.3377, 0.0000, 1.1503],
        [0.6

tensor([[1.2897, 0.6369, 0.3676,  ..., 0.3919, 0.0000, 1.3101],
        [0.8826, 0.3837, 0.2208,  ..., 0.2585, 0.0000, 1.0248],
        [0.8422, 0.3693, 0.1913,  ..., 0.1971, 0.0000, 0.8664],
        ...,
        [0.8717, 0.0000, 0.0186,  ..., 0.0000, 0.0000, 0.7171],
        [0.8476, 0.4451, 0.0000,  ..., 0.0000, 0.0000, 0.6498],
        [1.4790, 1.0152, 0.5549,  ..., 0.5345, 0.0000, 1.4016]],
       grad_fn=<ReluBackward0>)
tensor([[2.6175e-01, 8.7724e-01, 5.9003e-01,  ..., 6.6262e-01, 0.0000e+00,
         3.3775e-01],
        [1.2458e+00, 6.7845e-01, 6.8129e-01,  ..., 4.3726e-01, 0.0000e+00,
         1.2678e+00],
        [5.3920e-01, 9.1366e-01, 6.5745e-01,  ..., 6.5855e-01, 0.0000e+00,
         6.0083e-01],
        ...,
        [2.8511e+00, 1.2484e+00, 6.0580e-01,  ..., 4.1669e-01, 0.0000e+00,
         2.9174e+00],
        [1.6157e+00, 2.3157e-01, 9.4546e-02,  ..., 0.0000e+00, 0.0000e+00,
         1.5159e+00],
        [4.3003e-01, 1.7796e-01, 1.0942e-01,  ..., 1.3495e-03, 0.0000e+0

tensor([[1.5422, 0.6915, 0.4204,  ..., 0.4287, 0.0000, 1.6039],
        [1.0267, 0.4466, 0.2896,  ..., 0.3004, 0.0000, 1.2005],
        [0.9463, 0.4611, 0.2844,  ..., 0.2864, 0.0000, 0.9968],
        ...,
        [1.0309, 0.0000, 0.0336,  ..., 0.0000, 0.0000, 0.8959],
        [1.0178, 0.4650, 0.0000,  ..., 0.0000, 0.0000, 0.8413],
        [1.7521, 1.1108, 0.6548,  ..., 0.6185, 0.0000, 1.7208]],
       grad_fn=<ReluBackward0>)
tensor([[0.3340, 0.9896, 0.6973,  ..., 0.7706, 0.0000, 0.4393],
        [1.5384, 0.6879, 0.6774,  ..., 0.4364, 0.0000, 1.5973],
        [0.6596, 1.0271, 0.7781,  ..., 0.7607, 0.0000, 0.7474],
        ...,
        [3.2085, 1.4684, 0.8448,  ..., 0.6497, 0.0000, 3.3621],
        [1.8658, 0.2941, 0.1667,  ..., 0.0000, 0.0000, 1.8073],
        [0.4863, 0.2259, 0.1601,  ..., 0.0628, 0.0000, 0.4377]],
       grad_fn=<ReluBackward0>)
tensor([[1.1690, 0.5480, 0.2895,  ..., 0.4943, 0.0000, 1.2509],
        [2.3865, 2.2328, 1.3612,  ..., 1.0264, 0.0000, 2.5111],
        [1.0

...

tensor([[1.5931, 0.7814, 0.4968,  ..., 0.5083, 0.0000, 1.6797],
        [1.1259, 0.4490, 0.2870,  ..., 0.2854, 0.0000, 1.3100],
        [0.9865, 0.5218, 0.3412,  ..., 0.3377, 0.0000, 1.0502],
        ...,
        [1.0591, 0.0000, 0.1188,  ..., 0.0870, 0.0000, 0.9431],
        [1.0850, 0.5001, 0.0000,  ..., 0.0000, 0.0000, 0.9218],
        [1.8988, 1.1466, 0.6864,  ..., 0.6347, 0.0000, 1.8884]],
       grad_fn=<ReluBackward0>)
tensor([[0.3803, 1.0428, 0.7442,  ..., 0.8208, 0.0000, 0.5006],
        [1.6800, 0.6907, 0.6673,  ..., 0.4149, 0.0000, 1.7546],
        [0.7159, 1.0985, 0.8498,  ..., 0.8227, 0.0000, 0.8149],
        ...,
        [3.3990, 1.5379, 0.9109,  ..., 0.7036, 0.0000, 3.5848],
        [1.9123, 0.4013, 0.2722,  ..., 0.0746, 0.0000, 1.8817],
        [0.5225, 0.2522, 0.1883,  ..., 0.0885, 0.0000, 0.4804]],
       grad_fn=<ReluBackward0>)
tensor([[1.2205, 0.5924, 0.3325,  ..., 0.5274, 0.0000, 1.3129],
        [2.5534, 2.2984, 1.4144,  ..., 1.0513, 0.0000, 2.7034],
        [1.1

tensor([[1.6002, 0.7868, 0.4924,  ..., 0.5049, 0.0000, 1.6879],
        [1.1453, 0.4412, 0.2733,  ..., 0.2727, 0.0000, 1.3293],
        [0.9898, 0.5299, 0.3460,  ..., 0.3396, 0.0000, 1.0538],
        ...,
        [1.0574, 0.0000, 0.1356,  ..., 0.1049, 0.0000, 0.9431],
        [1.0862, 0.5114, 0.0000,  ..., 0.0000, 0.0000, 0.9248],
        [1.9164, 1.1512, 0.6860,  ..., 0.6324, 0.0000, 1.9070]],
       grad_fn=<ReluBackward0>)
tensor([[0.3874, 1.0477, 0.7456,  ..., 0.8249, 0.0000, 0.5086],
        [1.6830, 0.7020, 0.6716,  ..., 0.4175, 0.0000, 1.7581],
        [0.7200, 1.1123, 0.8617,  ..., 0.8328, 0.0000, 0.8200],
        ...,
        [3.4128, 1.5544, 0.9152,  ..., 0.7078, 0.0000, 3.6009],
        [1.9216, 0.4094, 0.2774,  ..., 0.0754, 0.0000, 1.8922],
        [0.5220, 0.2611, 0.1978,  ..., 0.0953, 0.0000, 0.4803]],
       grad_fn=<ReluBackward0>)
tensor([[1.2250, 0.6005, 0.3370,  ..., 0.5297, 0.0000, 1.3176],
        [2.5897, 2.2898, 1.3954,  ..., 1.0216, 0.0000, 2.7396],
        [1.1

tensor([[1.6011e+00, 7.9071e-01, 4.8357e-01,  ..., 4.9880e-01, 0.0000e+00,
         1.6896e+00],
        [1.1578e+00, 4.3506e-01, 2.5969e-01,  ..., 2.6106e-01, 0.0000e+00,
         1.3417e+00],
        [9.8879e-01, 5.3717e-01, 3.4848e-01,  ..., 3.3992e-01, 0.0000e+00,
         1.0527e+00],
        ...,
        [1.0590e+00, 1.2945e-03, 1.4186e-01,  ..., 1.1100e-01, 0.0000e+00,
         9.4542e-01],
        [1.0899e+00, 5.1521e-01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         9.2940e-01],
        [1.9209e+00, 1.1593e+00, 6.8769e-01,  ..., 6.3281e-01, 0.0000e+00,
         1.9122e+00]], grad_fn=<ReluBackward0>)
tensor([[0.3874, 1.0544, 0.7480,  ..., 0.8304, 0.0000, 0.5094],
        [1.6879, 0.7046, 0.6644,  ..., 0.4094, 0.0000, 1.7629],
        [0.7232, 1.1199, 0.8662,  ..., 0.8356, 0.0000, 0.8240],
        ...,
        [3.4228, 1.5612, 0.9041,  ..., 0.6975, 0.0000, 3.6124],
        [1.9270, 0.4139, 0.2775,  ..., 0.0711, 0.0000, 1.8982],
        [0.5207, 0.2682, 0.2042,  ..., 0.0993,

tensor([[1.5997, 0.7937, 0.4725,  ..., 0.4913, 0.0000, 1.6885],
        [1.1643, 0.4322, 0.2487,  ..., 0.2528, 0.0000, 1.3482],
        [0.9872, 0.5427, 0.3479,  ..., 0.3384, 0.0000, 1.0507],
        ...,
        [1.0605, 0.0100, 0.1443,  ..., 0.1134, 0.0000, 0.9473],
        [1.0933, 0.5167, 0.0000,  ..., 0.0000, 0.0000, 0.9335],
        [1.9251, 1.1628, 0.6834,  ..., 0.6281, 0.0000, 1.9165]],
       grad_fn=<ReluBackward0>)
tensor([[0.3888, 1.0575, 0.7454,  ..., 0.8312, 0.0000, 0.5112],
        [1.6954, 0.7012, 0.6489,  ..., 0.3950, 0.0000, 1.7700],
        [0.7259, 1.1251, 0.8673,  ..., 0.8358, 0.0000, 0.8271],
        ...,
        [3.4337, 1.5602, 0.8818,  ..., 0.6789, 0.0000, 3.6240],
        [1.9302, 0.4164, 0.2751,  ..., 0.0643, 0.0000, 1.9017],
        [0.5214, 0.2720, 0.2067,  ..., 0.0990, 0.0000, 0.4797]],
       grad_fn=<ReluBackward0>)
tensor([[1.2294, 0.6094, 0.3342,  ..., 0.5256, 0.0000, 1.3217],
        [2.6234, 2.2770, 1.3533,  ..., 0.9669, 0.0000, 2.7731],
        [1.2

tensor([[1.5966, 0.7964, 0.4609,  ..., 0.4841, 0.0000, 1.6850],
        [1.1664, 0.4319, 0.2407,  ..., 0.2479, 0.0000, 1.3502],
        [0.9847, 0.5473, 0.3467,  ..., 0.3367, 0.0000, 1.0475],
        ...,
        [1.0610, 0.0179, 0.1461,  ..., 0.1153, 0.0000, 0.9477],
        [1.0944, 0.5187, 0.0000,  ..., 0.0000, 0.0000, 0.9350],
        [1.9268, 1.1658, 0.6791,  ..., 0.6240, 0.0000, 1.9183]],
       grad_fn=<ReluBackward0>)
tensor([[0.3889, 1.0604, 0.7429,  ..., 0.8322, 0.0000, 0.5115],
        [1.6961, 0.7023, 0.6385,  ..., 0.3863, 0.0000, 1.7704],
        [0.7263, 1.1306, 0.8688,  ..., 0.8372, 0.0000, 0.8277],
        ...,
        [3.4374, 1.5628, 0.8633,  ..., 0.6651, 0.0000, 3.6276],
        [1.9280, 0.4216, 0.2759,  ..., 0.0608, 0.0000, 1.8994],
        [0.5217, 0.2749, 0.2083,  ..., 0.0981, 0.0000, 0.4800]],
       grad_fn=<ReluBackward0>)
tensor([[1.2296, 0.6132, 0.3319,  ..., 0.5234, 0.0000, 1.3215],
        [2.6284, 2.2746, 1.3369,  ..., 0.9454, 0.0000, 2.7779],
        [1.2

In [15]:
def predict(text):
    model.eval()
    with torch.no_grad():
        test_vector = torch.LongTensor(data.cv.transform([text]).toarray())

        output = model(test_vector)
        prediction = torch.sigmoid(output).item()

        if prediction > 0.5:
            print(f'{prediction:0.3}: Positive sentiment')
        else:
            print(f'{prediction:0.3}: Negative sentiment')

In [16]:
#predicting the sentence
test_text='It was a wonderful thing to watch! I would to love to see it again!!'
predict(test_text)

tensor([[0.1510, 0.0074, 0.0059, 0.0824, 0.0444, 0.1067, 0.1020, 0.0891, 0.1467,
         0.1010, 0.1321, 0.0000, 0.0000, 0.0136, 0.0499, 0.1098, 0.0686, 0.0613,
         0.1042, 0.0000, 0.0000, 0.0402, 0.1098, 0.0778, 0.1040, 0.0826, 0.0822,
         0.0865, 0.0775, 0.1040, 0.0105, 0.0989, 0.0000, 0.1581, 0.0493, 0.1019,
         0.0976, 0.0901, 0.0996, 0.0175, 0.1666, 0.0000, 0.0000, 0.0157, 0.0902,
         0.0058, 0.1340, 0.0316, 0.0000, 0.0000, 0.1825, 0.1172, 0.0780, 0.0706,
         0.0000, 0.0000, 0.1323, 0.0000, 0.1248, 0.0922, 0.1171, 0.0655, 0.1589,
         0.0000, 0.0968, 0.0000, 0.0000, 0.0264, 0.0000, 0.0264, 0.1493, 0.0000,
         0.1545, 0.0304, 0.1395, 0.0676, 0.0978, 0.0071, 0.0831, 0.1261, 0.0260,
         0.2259, 0.1405, 0.1551, 0.0244, 0.0021, 0.0000, 0.0730, 0.1074, 0.0870,
         0.1294, 0.0735, 0.0000, 0.0275, 0.1656, 0.0032, 0.0081, 0.1868, 0.0000,
         0.1410, 0.1119, 0.0000, 0.0182, 0.1118, 0.0852, 0.0036, 0.0000, 0.1206,
         0.0454, 0.0000, 0.0